In [1]:
import os

In [2]:
%pwd

'c:\\Projects\\Kidney_Disease_Classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Projects\\Kidney_Disease_Classification'

In [9]:
ML_FLOW_TRACKING_URI = 'https://dagshub.com/aayushkataria123/Kidney_Disease_Classification'
ML_FLOW_TRACKING_USERNAME = 'aayushkataria123'
ML_FLOW_TRACKING_PASSWORD = 'Webdevlopment'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [8]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model='artifacts/training/model.h5',
            training_data='artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone',
            all_params=self.params,
            mlflow_uri="https://dagshub.com/aayushkataria123/Kidney_Disease_Classification",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [11]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
import dagshub
dagshub.init(repo_owner='aayushkataria123', repo_name='Kidney_Disease_Classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Projects\Kidney_Disease_Classification\venv\Lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=6edd7891-bc29-47a8-ab73-3e54cdc43bc8&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=5d228c786f257670dc4bcbb5e94948f2fa59c51f4d82aac8e5800142a9808590




Accessing as aayushkataria123

Initialized MLflow to track repo "aayushkataria123/Kidney_Disease_Classification"

Repository aayushkataria123/Kidney_Disease_Classification initialized!

In [13]:
import tensorflow as tf

In [14]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [15]:
from dataclasses import dataclass
from pathlib import Path
@dataclass

class EvaluationConfig:
    path_of_mdel: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [16]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_mdel = Path('artifacts/training/model.h5'),
            training_data = Path('artifacts\data ingestion\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'),
            all_params = self.params,
            mlflow_uri = 'https://dagshub.com/aayushkataria123/Kidney_Disease_Classification',
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        
        )
        return eval_config

In [18]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config



    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,  # Normalize pixel values to [0, 1]
            validation_split=0.3  # Split data into training and validation sets
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )
        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self._valid_generator= valid_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    



    def evaluation(self):
        self.model=self.load_model(self.config.path_of_mdel)
        self._valid_generator()
        self.score = model.evaluate(self._valid_generator)
        self.save_score()

    def save_score(self):
        score = {'loss':self.score[0], 'accuracy':self.score[1]}
        save_json(Path('artifacts/evaluation/score.json'),data=score)
        
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({'loss':self.score[0], 'accuracy':self.score[1]})

            if tracking_url_type_store != 'file':

                mlflow.keras.log_model(self.model, 'model',registered_model_name='VGG16MODEL')
            else:
                mlflow.keras.log_model(self.model, 'model')

           

        

     

In [20]:
try:
    config= ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation=Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e



[2025-06-16 23:38:47,862 : INFO  :common : yaml file: config\config.yaml loaded successfully]
[2025-06-16 23:38:47,868 : INFO  :common : yaml file: params.yaml loaded successfully]
[2025-06-16 23:38:47,870 : INFO  :common : created directory at: artifacts]
Found 3725 images belonging to 4 classes.
233/233 [==============================] - 446s 2s/step - loss: 3.8377 - accuracy: 0.7034


2025/06/16 23:46:15 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-06-16 23:46:17,975 : WARNING  :save : Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\ayush\AppData\Local\Temp\tmpaazwndc7\model\data\model\assets
[2025-06-16 23:46:18,693 : INFO  :builder_impl : Assets written to: C:\Users\ayush\AppData\Local\Temp\tmpaazwndc7\model\data\model\assets]


c:\Projects\Kidney_Disease_Classification\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
